In [2]:
import pandas as pd
from datetime import datetime
import locale

# Chargement des données depus les fichiers CSV sources

lineScoreInput = pd.read_csv('line_score.csv', delimiter= ',')
def filter_line_scores_by_date(df, start_date, end_date):
    mask = (df['game_date_est'] >= end_date) & (df['game_date_est'] <= start_date)
    return df[mask]

lineScoreFiltered = filter_line_scores_by_date(
    lineScoreInput,
    '2015-04-18',
    '2014-10-28'
)

# Supposons que lineScoreFiltered est déjà chargé
df = lineScoreFiltered.copy()

# Convertir la colonne de date
df['game_date_est'] = pd.to_datetime(df['game_date_est'])

# Créer la table dates
dates = df[['game_date_est']].drop_duplicates().reset_index(drop=True)
dates['timestamp'] = dates['game_date_est'].apply(lambda x: x.timestamp()) #PK
dates['day'] = dates['game_date_est'].dt.day
dates['month'] = dates['game_date_est'].dt.month
dates['year'] = dates['game_date_est'].dt.year

dates = dates[['timestamp', 'day', 'month', 'year']]

# 🔹 3. Création de la table de faits game_data
# On garde uniquement les colonnes nécessaires
df['timestamp'] = df['game_date_est'].apply(lambda x: x.timestamp())  # PK
df.drop(columns=['game_date_est', 'team_id_home', 'team_id_away'], inplace=True)
game_data = df.copy()

nba_teams = pd.read_csv('NBA_Teams.csv')  # contient team_abbreviation, stats
city = pd.read_csv('cities_unique.csv')  # contient city
team_history = pd.read_csv('team_history.csv') 

dim_teams_temp = team_history.merge(
    city,
    left_on='city',
    right_on='city',
    how='left'
)

dim_teams_temp['team'] = dim_teams_temp['city'] + ' ' + dim_teams_temp['nickname']

# Jointure entre les deux tables qui portent des noms de colonnes différents mais qui présentent le même sens
dim_teams = nba_teams.merge(
    dim_teams_temp,
    left_on='TEAM',
    right_on='team',
    how='left'
)   

dim_teams.drop(columns=['team_id', 'team'], inplace=True)

playersStatsInput = pd.read_csv('players_stats.csv', delimiter= ',')
nba_salaire = pd.read_csv('nba-salaries.csv', delimiter= ',')

# 2. Garder uniquement les salaires de la saison 2015
nba_salaire_2015 = nba_salaire[nba_salaire['season'] == 2015]

# 3. Merge sur le nom du joueur
dim_players = playersStatsInput.merge(
    nba_salaire_2015[['name', 'salary', 'rank']],
    left_on='Name',
    right_on='name',
    how='left'
)
dim_players.drop(columns=['name'], inplace=True)
# 4. Renommer les colonnes pour clarté
dim_players = dim_players.rename(columns={
    'Name': 'player_name',
    'Team': 'team_abbreviation'
})

In [ ]:
game_data.to_csv('TP/game_data.csv', index=False)
dim_players.to_csv('TP/dim_players.csv', index=False)
dim_teams.to_csv('TP/dim_teams.csv', index=False)
dates.to_csv('TP/dates.csv', index=False)


     player_name  Games Played   MIN   PTS  FGM  FGA   FG%  3PM  3PA   3P%  \
0       AJ Price            26   324   133   51  137  37.2   15   57  26.3   
1   Aaron Brooks            82  1885   954  344  817  42.1  121  313  38.7   
2   Aaron Gordon            47   797   243   93  208  44.7   13   48  27.1   
3  Adreian Payne            32   740   213   91  220  41.4    1    9  11.1   
4     Al Horford            76  2318  1156  519  965  53.8   11   36  30.6   

   ...           Birthdate                    Collage  Experience  Height  \
0  ...     October 7, 1986  University of Connecticut           5   185.0   
1  ...    January 14, 1985       University of Oregon           6   180.0   
2  ...  September 16, 1995      University of Arizona           R   202.5   
3  ...   February 19, 1991  Michigan State University           R   205.0   
4  ...        June 3, 1986      University of Florida           7   205.0   

   Pos  team_abbreviation  Weight        BMI      salary   rank  
0 

In [3]:
# Ajouter les infos sur les équipes domicile
game_data = game_data.merge(
    dim_teams.rename(columns={'TEAM_ABBV': 'team_abbreviation_home'}),
    on='team_abbreviation_home',
    how='left',
    suffixes=('', '_home')
)

# Ajouter les infos sur les équipes à l'extérieur
game_data = game_data.merge(
    dim_teams.rename(columns={'TEAM_ABBV': 'team_abbreviation_away'}),
    on='team_abbreviation_away',
    how='left',
    suffixes=('', '_away')
)

# Ajouter la date
game_data = game_data.merge(
    dates,
    on='timestamp',
    how='left'
)

game_data = game_data.merge(
    dim_players.rename(columns={'player_name': 'player_name_home', 'team_abbreviation': 'team_abbreviation_home'}),
    on='team_abbreviation_home',
    how='left'
)


In [7]:
print(game_data['player_name_home'].head())
game_data.to_csv('TP/game_data_with_teams_and_players.csv', index=False)


0    Aron Baynes
1    Austin Daye
2     Boris Diaw
3    Cory Joseph
4    Danny Green
Name: player_name_home, dtype: object
